# Lab 1. Basic Techniques

## Regex

Expresiile regulate. Utile pentru cautare si normalizare de texte - https://www.w3schools.com/python/python_regex.asp


In [3]:
from sys import platform, path
if platform == "linux" or platform == "linux2":
    path.append('/home/dariusbuhai/python/lib/python3.9/site-packages')
import re

text = """
Praise for The Rain in Portugal
 
“Nothing in Billy Collins’s twelfth book . . . is exactly what readers might expect, and that’s the charm of this collection.”—The Washington Post
 
“This new collection shows [Collins] at his finest. . . . Certain to please his large readership and a good place for readers new to Collins to begin.”—Library Journal. 
 
“Disarmingly playful and wistfully candid.”—Booklist
Buy new:$38.65
No Import Fees Deposit & $13.01 Shipping to Romania Details -12.3.
"""

Sterge toate caracterele diferite de litere mari si mici ale alfabetului englez, apoi normalizeaza toate secventele de caractere de tip spatii la 1 spatiu

In [4]:
cleaned_text = re.sub("[^A-Za-z]", " ", text)
cleaned_text = re.sub("\s+", " ", cleaned_text)
print(cleaned_text)

 Praise for The Rain in Portugal Nothing in Billy Collins s twelfth book is exactly what readers might expect and that s the charm of this collection The Washington Post This new collection shows Collins at his finest Certain to please his large readership and a good place for readers new to Collins to begin Library Journal Disarmingly playful and wistfully candid Booklist Buy new No Import Fees Deposit Shipping to Romania Details 


Le putem testa cel mai usor pe https://pythex.org/ . Aici cautam toate numerele float sau int, impreuna cu pozitiile si valorile lor.

In [5]:
pattern = re.compile("[+-]?(\d+\.)?\d+")
for matching in pattern.finditer(text):
    print(matching.start(), matching.group())

418 38.65
450 13.01
484 -12.3


## Encodings

Encodarea unui text poate varia, in functie de limba si este importanta. Python foloseste standardul 'utf8'. Urmatorul exemplu este preluat dintr-o subtitrare (.srt) din limba rusa, dar nu este encodat in utf8. Citindu-l obtinem urmatoarea eroare:

In [6]:
with open('encoded_text.txt', "r") as fin:
    content = fin.read()
    print(content)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xdd in position 34: invalid continuation byte

Incercam sa detectam encoding-ul lui folosind chardet

In [ ]:
! pip install chardet

In [5]:
import chardet

with open('encoded_text.txt', "rb") as f:
    rawdata = f.read()
    result = chardet.detect(rawdata)
    extracted_encoding = result['encoding']
    print(extracted_encoding)

windows-1251


Reluam citirea si salvam continutul in format utf8

In [6]:
with open('encoded_text.txt', 'r', encoding=extracted_encoding) as fin:
    content = fin.read()
with open('utf8_text.txt', 'w', encoding='utf8') as fout:
    fout.write(content)

In [7]:
with open('utf8_text.txt', "r") as fin:
    content = fin.read()
    print(content)

1
00:00:05,100 --> 00:00:10,860
Это были тяжелые времена. Рим находился под господством коррумпированного Папы и сомнительных законов

2
00:00:10,960 --> 00:00:13,820
игр власти и междоусобной борьбы

3
00:00:15,660 --> 00:00:20,310
Синьоры начинали жестокие сражения с единственной целью – накопить состояние

4
00:00:21,640 --> 00:00:24,950
А тем временем, простые люди ели не каждый день

5
00:00:30,090 --> 00:00:36,040
Любовь была темой для поэтов, но редко упоминалась в свадебных клятвах

6
00:00:36,940 --> 00:00:42,140
Женщин отдавали в жены мужчинам, которых они едва знали, не говоря уже о любви

7
00:00:43,040 --> 00:00:47,980
В этом мире, жестоком и несправедливом, я повстречала двух молодых людей



## Non-standard files (PDF, Word, etc.)

Putem citi texte din documente word folosind:

In [ ]:
! pip install docx2txt

In [8]:
import docx2txt
my_text = docx2txt.process("soup.docx")
print(my_text)

Beautiful Soup is a Python library for pulling data out of HTML and XML files. It works with your favorite parser to provide idiomatic ways of navigating, searching, and modifying the parse tree. It commonly saves programmers hours or days of work.



These instructions illustrate all major features of Beautiful Soup 4, with examples. I show you what the library is good for, how it works, how to use it, how to make it do what you want, and what to do when it violates your expectations.



This document covers Beautiful Soup version 4.10.0. The examples in this documentation were written for Python 3.8.



You might be looking for the documentation for Beautiful Soup 3. If so, you should know that Beautiful Soup 3 is no longer being developed and that all support for it was dropped on December 31, 2020. If you want to learn about the differences between Beautiful Soup 3 and Beautiful Soup 4, see Porting code to BS4.



This documentation has been translated into other languages by Beaut

Sau pdf-uri care sunt salvate ca texte (nu poze):

In [ ]:
! pip install pdfplumber

In [9]:
import pdfplumber
with pdfplumber.open('soup.pdf') as pdf:
    for page in pdf.pages:
        print(page.extract_text())

Beautiful Soup is a Python library for pulling data out of HTML and XML files. It works with your favorite 
parser to provide idiomatic ways of navigating, searching, and modifying the parse tree. It commonly 
saves programmers hours or days of work. 
 
These instructions illustrate all major features of Beautiful Soup 4, with examples. I show you what the 
library is good for, how it works, how to use it, how to make it do what you want, and what to do when 
it violates your expectations. 
 
This document covers Beautiful Soup version 4.10.0. The examples in this documentation were written 
for Python 3.8. 
 
You might be looking for the documentation for Beautiful Soup 3. If so, you should know that Beautiful 
Soup 3 is no longer being developed and that all support for it was dropped on December 31, 2020. If 
you want to learn about the differences between Beautiful Soup 3 and Beautiful Soup 4, see Porting 
code to BS4. 
 
This documentation has been translated into other languages 

## Web scraping

Prin scraping ne referim la o multime de metode prin care putem descarca date nestructurate din mediul web, pe care le putem apoi procesa si stoca sub forma structurata.

Ca prim exemplu de scraping vom incerca urmatorul task: pornind de la site-ul de programare competitiva "infoarena.ro" dorim pentru un utilizator sa descarcam informatii despre toate submisiile efectuate de acesta.

Exemplu pagina de submisii: https://www.infoarena.ro/monitor?user=iordache.bogdan

Pentru a realiza un request care sa intoarca continutul paginii putem folosi biblioteca "requests":

In [ ]:
! pip install requests

In [10]:
import requests

def get_submissions_page(user):
    return requests.get(f"https://www.infoarena.ro/monitor?user={user}")

In [11]:
html = get_submissions_page("iordache.bogdan").content

Observam ca folosind metoda de mai sus putem descarca intreg continutul HTML al paginii. Pentru a extrage informatii utile trebuie sa parsam acest continut. Vom folosi biblioteca [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/):

In [12]:
import bs4

def parse_html(html):
    return bs4.BeautifulSoup(html, "html.parser")

Avand continutul parsat, putem determina acum cate submisii are in total acest utilizator:

In [13]:
import re

soup = parse_html(html)

# cautam un span care are clasa "count", in acest span se afla numarul de submisii
submission_count_text = soup.find("span", class_="count").text
print(submission_count_text)
submission_count = int(re.search(r"\d+", submission_count_text).group())
print(submission_count)

 (5027 rezultate)
5027


Observam ca aceste submisii sunt impartite in mai multe pagini (paginarea rezultatelor). Observam ca link-ul urmator: https://www.infoarena.ro/monitor?user=iordache.bogdan&display_entries=250&first_entry=100 ne returneaza 250 de submisii, incepand cu submisia cu numarul 100. Modificam metoda get_submissions_page astfel:

In [14]:
def get_submissions_page(user, display_entries=None, first_entry=None):
    req_string = f"https://www.infoarena.ro/monitor?user={user}"
    if display_entries is not None:
        req_string += f"&display_entries={display_entries}"
    if first_entry is not None:
        req_string += f"&first_entry={first_entry}"

    return requests.get(req_string)

Implementam o functie care returneaza informatii despre toate submisiile unui utilizator:

In [15]:
from tqdm import tqdm
import pandas as pd
import pdb

def scrape_submissions(user):
    # determinam numarul total de submisii
    html = get_submissions_page(user).content
    soup = parse_html(html)
    submission_count_text = soup.find("span", class_="count").text
    submission_count = int(re.search(r"\d+", submission_count_text).group())

    # vom salva in acest dictionar datele despre submisiile extrase, structura aceasta
    # ne va ajuta ulterior sa construim un tabel (dataframe) folosind pandas
    d = {
        "id": [],
        "problema": [],
        "url_problema": [],
        "url_sursa": [],
        "data": [],
        "puncte": [],
    }

    # accesam pagini cu submisii in grupuri de 250
    for first_entry in tqdm(range(0, submission_count, 250)):
        html = get_submissions_page(user, display_entries=250, first_entry=first_entry).content
        soup = parse_html(html)

        # selectam toate liniile de tabel (tr)
        lines = soup.select("table.monitor tbody tr")

        for line in lines:
            # selectam celulele de pe aceasta linie
            cells = [cell for cell in line.select("td")]

            # extragem link-urile pentru problema si codul sursa
            try:
                url_problema = cells[2].select_one("a")["href"]
                url_sursa = cells[4].select_one("a")["href"]
            except Exception:  # daca vreun link nu exista ignoram linia
                continue
            
            d["id"].append(cells[0].text)
            d["problema"].append(cells[2].text)
            d["url_problema"].append(url_problema)
            d["url_sursa"].append(url_sursa)
            d["data"].append(cells[5].text)

            try:
                puncte = int(re.search(r"\d+", cells[6].text).group())
            except Exception:
                puncte = 0
            d["puncte"].append(puncte)

    return pd.DataFrame(d)

In [16]:
df_submissions = scrape_submissions("iordache.bogdan")

100%|███████████████████████████████████████████████████| 21/21 [00:10<00:00,  2.04it/s]


In [17]:
df_submissions.head()

,id,problema,url_problema,url_sursa,data,puncte
0,#2836860,Luff,/problema/luff,/job_detail/2836860?action=view-source,21 ian 22 00:50:03,100
1,#2836859,Luff,/problema/luff,/job_detail/2836859?action=view-source,21 ian 22 00:49:40,0
2,#2836858,Luff,/problema/luff,/job_detail/2836858?action=view-source,21 ian 22 00:41:56,70
3,#2836856,Luff,/problema/luff,/job_detail/2836856?action=view-source,21 ian 22 00:36:59,70
4,#2836855,Luff,/problema/luff,/job_detail/2836855?action=view-source,21 ian 22 00:36:26,70


In [19]:
df_submissions.to_csv("submissions.csv", index=False)

Exemplu scriere/citire fisier JSON:

In [20]:
import json

vec = [
    {"title": "example_1", "size": 7},
    {"title": "example_2", "size": 3},
    {"title": "example_3", "size": 8},
]

with open("example.json", "w") as f:
    json.dump(vec, f, indent=4)

In [21]:
with open("example.json", "r") as f:
    vec = json.load(f)
print(vec)

[{'title': 'example_1', 'size': 7}, {'title': 'example_2', 'size': 3}, {'title': 'example_3', 'size': 8}]


Un alt mod de a face scraping este sa folosim biblioteca pandas pentru a ne extrage tabele html, transformandu-le in DataFrame-uri, pe care le putem manipula foarte usor. Un exemplu util este extragerea sarbatorile legale romanesti, in anul 2022, de pe https://www.timeanddate.com/.

In [ ]:
! pip install lxml

In [23]:
import pandas as pd

tables_df = pd.read_html('https://www.timeanddate.com/holidays/romania/2022?hol=1')
df = tables_df[0]

# Il putem curata prin a sterge liniile nule si modifica coloanele de la tuplul "(Date, Date)" -> "Date"
df = df.dropna(axis='index')
df.columns = ['Date', 'Day', 'Name', 'Type']

# Reindexam tabelul
df = df.reset_index(drop="True")

# Afisam primele 5 randuri
df.head()

,Date,Day,Name,Type
0,1 ian,sâmbătă,New Year's Day,National holiday
1,2 ian,duminică,Day after New Year's Day,National holiday
2,24 ian,luni,Unification Day,National holiday
3,19 feb,sâmbătă,Constantin Brancusi Day,Observance
4,24 feb,joi,Dragobete,Observance


In [25]:
# Daca vrem se vedem sarbatorile care pica in ziua de luni
df_luni = df.loc[df["Day"] == "luni"]
df_luni

,Date,Day,Name,Type
2,24 ian,luni,Unification Day,National holiday
10,25 apr,luni,Orthodox Easter Monday,"National holiday, Orthodox"
19,13 iun,luni,Orthodox Pentecost Monday,"National holiday, Orthodox"
23,15 aug,luni,St Mary's Day,National holiday
25,31 oct,luni,Halloween,Observance
33,26 dec,luni,Second day of Christmas,National holiday


In [26]:
df_luni.describe()

,Date,Day,Name,Type
count,6,6,6,6
unique,6,1,6,3
top,24 ian,luni,Unification Day,National holiday
freq,1,6,1,3


Putem salva rezultatul (la fel ca orice dictionar de python) intr-un json, ca alternativa la DataFrame - si poate fi util intr-o aplicatie pentru comunicarea cu front-end-ul

In [27]:
import json
json_str = df.to_json(orient='records')
json_result = json.loads(json_str)

with open('holidays.json', 'w', encoding='utf8') as fout:
    json.dump(json_result, fout, indent=4, sort_keys=True, ensure_ascii=False)

Alte biblioteci utile pentru scraping:
 * [scrapy](https://scrapy.org/) (folosit in special pentru web crawling)
 * [selenium](https://selenium-python.readthedocs.io/) (folosit pentru a simula activitatea din browser, utilizat in special in scrierea de teste pentru aplicatii front-end)

## TASK: IMDb scraping (deadline: 3 martie ora 23:59)

1. Pornind de la lista cu cele mai populare 250 de filme de pe IMDb ([https://www.imdb.com/chart/top/](https://www.imdb.com/chart/top/)), identificati pentru toate aceste filme link-ul catre pagina sa de recenzii.

Exemplu: aici se gaseste pagina cu recenzii pentru "The Shawshank Redemption": [https://www.imdb.com/title/tt0111161/reviews](https://www.imdb.com/title/tt0111161/reviews)

In [5]:
import bs4
import requests

def parse_html(html):
    return bs4.BeautifulSoup(html, "html.parser")

def parse_html_from_link(link):
    res = requests.get(link)
    return parse_html(res.content)

# Save all movies with their associated details, including reviews_links
movies = list()
html = parse_html_from_link("https://www.imdb.com/chart/top/")
tds = html.findAll("td", attrs="titleColumn")
for td in tds:
    a = td.find("a")
    link = a.attrs['href']
    movie_id = link.split("/")[2]
    reviews_link = f"https://www.imdb.com/title/{movie_id}/reviews"
    movies.append({
        "reviews_link": reviews_link,
        "movie": a.text,
        "movie_id": movie_id
    })
    print(f"Loaded movie: {a.text}. Id: {movie_id}. Reviews link: {reviews_link}")



Loaded movie: Închisoarea îngerilor. Id: tt0111161. Reviews link: https://www.imdb.com/title/tt0111161/reviews
Loaded movie: Nașul. Id: tt0068646. Reviews link: https://www.imdb.com/title/tt0068646/reviews
Loaded movie: Nașul: Partea a II-a. Id: tt0071562. Reviews link: https://www.imdb.com/title/tt0071562/reviews
Loaded movie: Cavalerul negru. Id: tt0468569. Reviews link: https://www.imdb.com/title/tt0468569/reviews
Loaded movie: 12 Oameni mânioşi. Id: tt0050083. Reviews link: https://www.imdb.com/title/tt0050083/reviews
Loaded movie: Lista lui Schindler. Id: tt0108052. Reviews link: https://www.imdb.com/title/tt0108052/reviews
Loaded movie: Stăpânul inelelor: Întoarcerea regelui. Id: tt0167260. Reviews link: https://www.imdb.com/title/tt0167260/reviews
Loaded movie: Pulp Fiction. Id: tt0110912. Reviews link: https://www.imdb.com/title/tt0110912/reviews
Loaded movie: Cel bun, cel rău, cel urât. Id: tt0060196. Reviews link: https://www.imdb.com/title/tt0060196/reviews
Loaded movie: Stă

2. Pentru fiecare film colectati date despre recenziile sale (titlu, text, rating, data, utlizator, etc.)

In [6]:
def parse_review_page(html_review):
    reviews_tags = html_review.findAll("div", attrs={"class": "imdb-user-review"})
    reviews = []
    for review_tag in reviews_tags:
        title = review_tag.find("a", attrs={"class": "title"}).text
        text = review_tag.find("div", attrs={"class": "text"}).text
        try:
            rating = review_tag.find("span", attrs={"class": "rating-other-user-rating"}).find("span").text
        except:
            rating = 0
        data = review_tag.find("span", attrs={"class": "review-date"}).text
        username = review_tag.find("span", attrs={"class": "display-name-link"}).find("a").text
        reviews.append({
            "title": title,
            "text": text,
            "data": data,
            "rating": rating,
            "username": username,
            "movie": movie['movie']
        })
    return reviews

reviews = list()
for movie in movies:
    html_review = parse_html_from_link(movie['reviews_link'])
    reviews.extend(parse_review_page(html_review))
    print(f"Saved movie reviews: {movie['movie']}")

Saved movie reviews: Închisoarea îngerilor
Saved movie reviews: Nașul
Saved movie reviews: Nașul: Partea a II-a
Saved movie reviews: Cavalerul negru
Saved movie reviews: 12 Oameni mânioşi
Saved movie reviews: Lista lui Schindler
Saved movie reviews: Stăpânul inelelor: Întoarcerea regelui
Saved movie reviews: Pulp Fiction
Saved movie reviews: Cel bun, cel rău, cel urât
Saved movie reviews: Stăpânul inelelor: Frăția inelului
Saved movie reviews: Fight Club - Sala de lupte
Saved movie reviews: Forrest Gump
Saved movie reviews: Începutul
Saved movie reviews: Stăpânul inelelor: Cele două turnuri
Saved movie reviews: Războiul stelelor - Episodul V: Imperiul contraatacă
Saved movie reviews: Matrix
Saved movie reviews: Băieți buni
Saved movie reviews: Zbor deasupra unui cuib de cuci
Saved movie reviews: Cei șapte samurai
Saved movie reviews: Se7en
Saved movie reviews: Tăcerea mieilor
Saved movie reviews: Orașul zeilor
Saved movie reviews: Viața e frumoasă
Saved movie reviews: O viață minunată


3. Creati un dataset de recenzii, pentru fiecare recenzie stocati:
 * filmul caruia ii apartine
 * titlul recenziei
 * textul recenziei
 * ratingul
 * data
 * utilizator

 Salvati datasetul intr-un fisier JSON.

In [ ]:
import json
with open("reviews.json", "w") as f:
    f.write(json.dumps(reviews))

4. Pe o pagina cu recenzii putem gasi un numar mic de astfel de date. Butonul de "Load more" de la final, cand este apasat,
produce un request care returneaza HTML-ul urmatoarelor recenzii.
Folosind aceasta logica colectati automat pentru fiecare film un numar mai mare de recenzii.

In [ ]:
reviews = list()
for movie in movies:
    html_review = parse_html_from_link(movie['reviews_link'])
    limit_pages = 5
    while html_review is not None and limit_pages > 0:
        reviews.extend(parse_review_page(html_review))
        pagination_key = html_review.find("div", attrs={"class": "load-more-data"}).attrs['data-key']
        html_review = parse_html_from_link(f"https://www.imdb.com/title/{movie['movie_id']}/reviews/_ajax?ref_=undefined&paginationKey={pagination_key}")
        limit_pages -= 1
    print(f"Saved movie extended reviews (5 pages): {movie['movie']}")

with open("reviews_extended.json", "w") as f:
    f.write(json.dumps(reviews))
